In [2]:
library(dplyr)
library(stringr)

# FINAL RULES FOR PRINCIPAL COMPONENT

In [3]:
list_no_events <- readRDS("data/list_no_events.rds")
factors <- c("ZRSR2","U2AF1_p.S34","U2AF1_p.Q157","SRSF2","SF3B1","ASXL1","STAG2","BCOR","RUNX1","EZH2","MLL","PHF6","SF1","NF1","CUX1","SETBP1")
Trisomies_list <- c("add_8","add_11","add_13","add_21","add_22")
deletions_list <- c('del_20','del_3','del_5','del_7','del_9','del_12','del_13','del_16','del_17','del_18','minusy')
# List patient with OS endpoint (2113)
list_patients_with_OS <- readRDS("data/list_patients_with_OS.rds")

df <- read.table("data/bdp_with_comp.tsv")

df <- df[list_patients_with_OS,]


In [4]:
# Patient IDS without any mutation (46)
list_no_events <- readRDS("data/list_no_events.rds")

# Patient IDS with complex karyotype, no TP53 and no deletions 
list_patients_complex_no_deletions_and_no_TP53 <- readRDS("data/list_patients_complex_no_deletions_and_no_TP53_training_cohort.rds")

# sAML class defining events
factors <- c("ZRSR2","U2AF1_p.S34","U2AF1_p.Q157","SRSF2","SF3B1","ASXL1","STAG2","BCOR","RUNX1","EZH2","MLL","PHF6","SF1","NF1","CUX1","SETBP1")

# class defining trisomies events
Trisomies_list <- c("add_8","add_11","add_13","add_21","add_22")

# Classification begins :
df$principal_component <- ""

df[df$NPM1==1,"principal_component"] <- "NPM1"

df[df$inv_16==1 & df$principal_component=="","principal_component"] <- "inv(16)"


df[df$t_8_21==1 & df$principal_component=="","principal_component"] <- "t(8;21)"

df[df$t_15_17==1 & df$principal_component=="","principal_component"] <- "t(15;17)"

df[(df$TP53==1 | df$complex==1) & !(rownames(df) %in% list_patients_complex_no_deletions_and_no_TP53) 
   & df$principal_component=="","principal_component"] <- "TP53-complex"

df[df$inv_3==1 & df$principal_component=="","principal_component"] <- "inv(3)"

df[ (df$t_9_11==1 | df$t_v_11==1) & df$principal_component=="","principal_component"] <- "t(11)"

df[df$t_6_9==1 & df$principal_component=="","principal_component"] <- "t(6;9)"

df[rowSums(df[,factors])>1 & df$principal_component=="","principal_component"] <- "sAML2"

df[df$CEBPA_bi==1 & df$principal_component=="","principal_component"] <- "CEBPA_bi"

df[rowSums(df[,factors])==1 & df$principal_component=="","principal_component"] <- "sAML1"

df[rowSums(df[,factors])==1 & df$principal_component=="sAML1" & (df$DNMT3A==1 | df$TET2==1) ,"principal_component"] <- "sAML2"

df[df$WT1==1 & df$principal_component=="","principal_component"] <- "WT1"

# one of main Trisomies (+8,+11,+13,+21,+22) OR patients that are complex, no TP53 and no deletions.
df[(rowSums(df[,Trisomies_list])>0 | rownames(df) %in% list_patients_complex_no_deletions_and_no_TP53)
   & df$principal_component=="","principal_component"] <- "Trisomies"

df[(df$DNMT3A==1 & (df$IDH1==1 | df$IDH2_p.R140==1 | df$IDH2_p.R172==1)) & df$principal_component=="","principal_component"] <- "DNMT3A-IDH"

df[rownames(df) %in% list_no_events,"principal_component"] <- "no events"

df[df$principal_component=="","principal_component"] <- "mNOS"

table(df$principal_component)


    CEBPA_bi   DNMT3A-IDH      inv(16)       inv(3)         mNOS    no events 
          38           19           92           17          124           46 
        NPM1        sAML1        sAML2        t(11)     t(15;17)       t(6;9) 
         674          100          501           75           19           16 
     t(8;21) TP53-complex    Trisomies          WT1 
         100          208           44           40 